In [1]:
import sys
sys.path.append("../")

In [2]:
statement_name = "../tmp/Справка_о_движении_денежных_средств_0819.pdf"
incomes = 9678606.76
expences = -9128682.41

# 1. Прочитать выписку

In [3]:
from src.finman.utils.pdf import extract_text_from_pdf
from src.finman.expenses.tbank import parse_operations

In [4]:
text = extract_text_from_pdf(statement_name)
transactions_df = parse_operations(text)

In [5]:
assert transactions_df.loc[transactions_df['Сумма операции в валюте карты'] > 0]['Сумма операции в валюте карты'].sum() == incomes
assert transactions_df.loc[transactions_df['Сумма операции в валюте карты'] < 0]['Сумма операции в валюте карты'].sum() == expences

In [6]:
transactions_df.shape

(1314, 9)

In [7]:
transactions_df.head()

,id,Дата операции,Время операции,Сумма операции в валюте карты,Валюта карты,Сумма в валюте операции,Валюта операции,Описание операции,Номер карты
0,de41e78861,2024-08-19,2024-08-19 12:45:00,-725.85,₽,-725.85,₽,Оплата в BRUSNIKA Gorod Moskva RUS,1665
1,a126bde1e1,2024-08-19,2024-08-19 09:06:00,-1103.00,₽,-1103.00,₽,Оплата в YANDEX*5411*LAVKA Moscow RUS,3685
2,72c5c2d74b,2024-08-18,2024-08-18 19:46:00,-197.00,₽,-197.00,₽,Оплата в BELKACAR MOSKVA RUS,5223
3,e655a37648,2024-08-18,2024-08-18 13:16:00,-640.00,₽,-640.00,₽,Внешний перевод по номеру телефона +79207894867,5037
4,d6cf48decc,2024-08-18,2024-08-18 13:12:00,-580.00,₽,-580.00,₽,Оплата в PTICHE MOLOKO MOSCOW RUS,1665


# 2. Запроцессить гугол

In [8]:
from src.finman.utils.gsheet import GSheetWorker

In [9]:
sheet_id = "1GqwNPuOtQUXfltK6F4oHpx-U1BqBgiDZfC5YfDcJyto"
tbank_main_spreadsheet_name = "t-bank (main)"
gcreds_file = "../secrets/finman-433017-da308d823497.json"

In [10]:
gsw = GSheetWorker(gcreds_file)

In [11]:
gs_df = gsw.get_df(sheet_id, tbank_main_spreadsheet_name)

In [12]:
gs_df.head()

,id,Дата операции,Время операции,Сумма операции в валюте карты,Валюта карты,Сумма в валюте операции,Валюта операции,Описание операции,Номер карты


In [13]:
missing_recrods = transactions_df.loc[~transactions_df['id'].isin(gs_df['id'])]

In [14]:
missing_recrods

,id,Дата операции,Время операции,Сумма операции в валюте карты,Валюта карты,Сумма в валюте операции,Валюта операции,Описание операции,Номер карты
0,de41e78861,2024-08-19,2024-08-19 12:45:00,-725.85,₽,-725.85,₽,Оплата в BRUSNIKA Gorod Moskva RUS,1665
1,a126bde1e1,2024-08-19,2024-08-19 09:06:00,-1103.00,₽,-1103.00,₽,Оплата в YANDEX*5411*LAVKA Moscow RUS,3685
2,72c5c2d74b,2024-08-18,2024-08-18 19:46:00,-197.00,₽,-197.00,₽,Оплата в BELKACAR MOSKVA RUS,5223
3,e655a37648,2024-08-18,2024-08-18 13:16:00,-640.00,₽,-640.00,₽,Внешний перевод по номеру телефона +79207894867,5037
4,d6cf48decc,2024-08-18,2024-08-18 13:12:00,-580.00,₽,-580.00,₽,Оплата в PTICHE MOLOKO MOSCOW RUS,1665
...,...,...,...,...,...,...,...,...,...
1309,8f2d71ef52,2024-01-02,2024-01-02 15:06:00,-305.67,₽,-305.67,₽,Оплата в CITYDRIVE Gorod Moskva RUS,2475
1310,6ccac9495b,2024-01-02,2024-01-02 11:44:00,-1600.00,₽,-1600.00,₽,Оплата в GOSUSLUGI.RU MOSKVA RUS,2475
1311,6b0565872a,2024-01-01,2024-01-01 17:42:00,-2000.00,₽,-2000.00,₽,Внутренний перевод на договор,5421
1312,ee2f8c8d22,2024-01-01,2024-01-01 03:31:00,-100.00,₽,-100.00,₽,Оплата в YANDEX*4121*TAXI Moskva RUS,3685


In [15]:
# missing_recrods['Время операции'].astype(str)

In [16]:
# missing_recrods['Дата и время операции'] = missing_recrods['Дата и время операции'].dt.strftime('%Y-%m-%d %H:%M')
# missing_recrods['Дата списания'] = missing_recrods['Дата списания'].dt.strftime('%Y-%m-%d %H:%M')
missing_recrods['Дата операции'] = missing_recrods['Дата операции'].astype(str)
missing_recrods['Время операции'] = missing_recrods['Время операции'].dt.strftime('%Y-%m-%d %H:%M')


gsw.insert_df(missing_recrods, sheet_id, tbank_main_spreadsheet_name)